# test report: comparison with tested data

In [ ]:
import os, sys
sys.path.insert(0, os.path.join("..", ".."))

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

In [ ]:
ref_data = pd.read_csv("reference_data/L2_reference_data_20220521T110815.csv", index_col=["time", "sample"], parse_dates=True).sort_index()
ref_data = gpd.GeoDataFrame(ref_data.drop(columns=["lon", "lat"]), geometry=gpd.points_from_xy(ref_data.lon, ref_data.lat), crs=4326)
ref_data["power"] = np.power(10, ref_data.power/10)

In [ ]:
from cryoswath import l1b

In [ ]:
l1b_data = l1b.from_id("20220521T110815", smooth_phase_difference=False, drop_outside=None, waveform_selection=ref_data.index.levels[0]).append_best_fit_phase_index()

In [ ]:
l2_swath, l2_poca = l1b_data.to_l2(swath_or_poca="both")

In [ ]:
ref_data.index = ref_data.index.set_levels(l2_swath.index.levels[0], level=0)

In [ ]:
ref_poca = ref_data[ref_data.rel_samp_nr==0]
ref_poca.index = ref_poca.index.get_level_values("time")

In [ ]:
ref_swath = ref_data[~(ref_data.rel_samp_nr<20)]

## swath elevation estimates

In [ ]:
(l2_swath.height-ref_swath.height).plot.hist(xlabel="elevation difference, meter");

The plot above ideally shows, that there are not many values outside of the bin that includes zero.

In [ ]:
((l2_swath.height-ref_swath.height)*100).plot.hist(bins=np.arange(-30.5, 30.6, 1), xlabel="elevation difference, centimeter");

The figure above shows the deviation in detail. Since the data and their reference were retrieved from the same raw data, the differences should be very small.

In [ ]:
distance = l2_swath.distance(ref_swath.to_crs(l2_swath.crs))

In [ ]:
distance.plot.hist(xlabel="distance, meter");

The above figure shows the horizontal distance between the compared
data. Again, the differences should be small. Differences can for
example occur if the phase ambiguity has been solved differently. For
those cases, we'll check later whether they look plausible.

In [ ]:
(distance*1e3).plot.hist(bins=np.linspace(0, 50, 25), xlabel="distance, millimeter");

As before, most differences should be small. The scale was initially
chosen to be millimeters to resolve the differences despite good
agreement. However, in the horizontal, even differences up to 100 m
would be reasonable, given the uncertainty of the phase difference.

In [ ]:
l2_swath[distance>1].h_diff.plot.hist(xlabel="elevation difference to DEM, meter", bins=100);

The above figure checks whether, in the cases where the data disagrees
on the phase ambiguity, the tested data looks reasonable. Here, the
elevation data is compared against the reference DEM.

## POCA elevation estimate.

Below, the test figures are repeated for POCAs.

*note:*  
There are substantial differences between the old and the current
POCA results. Checking the `l1b_swath_start.ipynb`, that also shows POCA
points, it seems that the current implementation is good. A source of
the discrepancy could well be involving non-glaciated areas. In future,
reference data that definitely are on glaciated areas should be used.

In [ ]:
(l2_poca.height-ref_poca.height).plot.hist(xlabel="elevation difference, meter", bins=100);

In [ ]:
(l2_poca.height-ref_poca.height).plot.hist(bins=np.arange(-30.5, 30.6, 1), xlabel="elevation difference, meter");

In [ ]:
distance = l2_poca.distance(ref_poca.to_crs(l2_poca.crs));

In [ ]:
distance.plot.hist(xlabel="distance, meter");

In [ ]:
distance.plot.hist(bins=np.linspace(0, 200, 25), xlabel="distance, meter");

In [ ]:
# How well do those points agree with the DEM that do not match the reference location?
l2_poca[distance>1].h_diff.plot.hist(xlabel="elevation difference to DEM, meter", bins=25);